In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# pip install git+https://github.com/microsoft/responsible-ai-toolbox-privacy.git#egg=privacy_estimates[pipelines]
from privacy_estimates.experiments.aml import JobList, Job

In [ ]:
job_urls = {
    "qwen-base": "https://ml.azure.com/experiments/id/50bdc468-77f2-46e4-8087-361a55de71c5/runs/gray_gold_mhrjx2kvc8?wsid=/subscriptions/acc09744-1ee3-4242-b375-93421c63af0c/resourceGroups/Singularity/providers/Microsoft.MachineLearningServices/workspaces/M365Research&tid=72f988bf-86f1-41af-91ab-2d7cd011db47#",
    "qwen-cot": "https://ml.azure.com/experiments/id/50bdc468-77f2-46e4-8087-361a55de71c5/runs/funny_chayote_d1mw56sxmd?wsid=/subscriptions/acc09744-1ee3-4242-b375-93421c63af0c/resourceGroups/Singularity/providers/Microsoft.MachineLearningServices/workspaces/M365Research&tid=72f988bf-86f1-41af-91ab-2d7cd011db47#",
    "qwen-rl": "https://ml.azure.com/runs/zen_rocket_b7rdw20cl4?wsid=/subscriptions/acc09744-1ee3-4242-b375-93421c63af0c/resourcegroups/Singularity/workspaces/M365Research&tid=72f988bf-86f1-41af-91ab-2d7cd011db47#",
}

In [ ]:
jobs = JobList.from_urls(job_urls.values())

In [ ]:
def postprocess(job: Job):
    metrics = job.get_node("privacylens").get_metrics()
    return metrics

In [ ]:
metrics = [postprocess(job) for job in jobs]

In [ ]:
categories = [
    'info_leakage_rate',
    'adjusted_info_leakage_rate',
    'average_helpfulness_score',
    'binary_helpfulness_rate',
]

In [ ]:
data = pd.DataFrame(metrics)
data["model"] = job_urls.keys()
data = data[["model"] + categories]

In [ ]:
data

In [ ]:
# Copy this to the results/privacylens.csv on overleaft to update results
data.to_csv("privacylens.csv", index=False)

In [ ]:
# Number of variables
N = len(categories)

# Compute angle for each axis in the plot (in radians)
angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
angles += angles[:1]  # complete the loop

# Create a polar subplot
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# Plot each experiment's metrics
for _, row in data.iterrows():
    # Get the values for the specified metrics, defaulting to 0 if a key is missing.
    values = [row[cat] for cat in categories]
    values += values[:1]  # complete the loop
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=row['model'])
    ax.fill(angles, values, alpha=0.25)

# Set the category labels
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

# Add a legend
plt.legend(loc='lower right')

plt.show()